##CA02: Email Spam Classifier (Naive Bayes)

This assignment task builds an email spam classifier using the Naive Bayes supervised machine learning algorithm.

In [ ]:
#Checking Directory Contents

import os
os.listdir('.')

['.config',
 'train-mails.zip',
 'CA02_NB_assignment.ipynb',
 'test-mails.zip',
 'sample_data']

This code is used to ensure that the required data folders are available before reading the email files.

In [ ]:
!ls /content

CA02_NB_assignment.ipynb  sample_data  test-mails.zip  train-mails.zip


The code help understand the data by checking the available files and folder structure before further processing.

In [ ]:
#Unzip Zip files

!unzip train-mails.zip
!unzip test-mails.zip

Archive:  train-mails.zip
   creating: train-mails/
  inflating: __MACOSX/._train-mails  
  inflating: train-mails/6-338msg1.txt  
  inflating: __MACOSX/train-mails/._6-338msg1.txt  
  inflating: train-mails/5-1300msg3.txt  
  inflating: __MACOSX/train-mails/._5-1300msg3.txt  
  inflating: train-mails/6-250msg1.txt  
  inflating: __MACOSX/train-mails/._6-250msg1.txt  
  inflating: train-mails/6-266msg3.txt  
  inflating: __MACOSX/train-mails/._6-266msg3.txt  
  inflating: train-mails/spmsgc10.txt  
  inflating: __MACOSX/train-mails/._spmsgc10.txt  
  inflating: train-mails/5-1326msg1.txt  
  inflating: __MACOSX/train-mails/._5-1326msg1.txt  
  inflating: train-mails/spmsga114.txt  
  inflating: __MACOSX/train-mails/._spmsga114.txt  
  inflating: train-mails/spmsga100.txt  
  inflating: __MACOSX/train-mails/._spmsga100.txt  
  inflating: train-mails/6-110msg3.txt  
  inflating: __MACOSX/train-mails/._6-110msg3.txt  
  inflating: train-mails/6-126msg1.txt  
  inflating: __MACOSX/train-ma

The "!" tells the notebook to run a shell (terminal) command instead of Python code.

*   The unzip command extracts the contents of a compressed ZIP file.
*   train-mails.zip is unzipped to make the training email data available for use in the current working directory.
*  test-mails.zip is unzipped to make the testing email data available for evaluating a model.

Together, these commands prepare the dataset so it can be accessed and processed in later steps of a data analysis or machine learning workflow.


In [ ]:
import os
os.listdir('.')

['.config',
 'train-mails.zip',
 'CA02_NB_assignment.ipynb',
 'train-mails',
 '__MACOSX',
 'test-mails.zip',
 'test-mails',
 'sample_data']

*   The import os statement loads Python’s built-in os module, which provides functions for interacting with the operating system.
*   The function call os.listdir('.') asks the operating system to list all files and folders in the current working directory.
*   The dot '.' represents the current directory where the notebook or script is running.

As a result, this line returns a list of filenames and folder names that exist in that directory, allowing you to see what files are available to work with.

**Main Takeaways:** train-mails and test-mails are accessible in the Output.



In [ ]:
train_dir = './train-mails'
test_dir = './test-mails'

Storing these paths in variables makes the code cleaner, easier to read and load, and simpler to reuse when loading or processing files from the training and test datasets later in the program.

##Import Required Libaries

It is required to prepare all the necessary libraries before building the Naive Bayes model.
*   In this section, all necessary Python libraries are imported.  
*   These libraries support file handling, numerical operations, text processing, machine learning model construction, and model evaluation.

In [ ]:
# Standard libraries
import os
import string
from collections import Counter

# Third-party libraries
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix


Standard Library Imports

This section imports Python’s standard libraries that are used for file handling and basic data processing.

- `os`: file and directory handling  
- `string`: text preprocessing  
- `Counter`: word frequency counting

In [ ]:
#Third-Party Libraries

import os
import numpy as np
from collections import Counter
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix

This section includes the libraries needed to load data, process text, build the Naive Bayes model, and evaluate its performance.

- `os` is used to access email files stored in folders.
- `numpy` supports basic numerical operations.
- `Counter` counts word frequencies in the emails.
- `MultinomialNB` is the Naive Bayes classifier used for spam detection.
- `accuracy_score` and `confusion_matrix` evaluate the model’s performance.

## Building the Vocabulary (Feature Engineering)

This function builds a dictionary of the most frequent words from the training email dataset.
It reads all emails in the training folder, extracts words, removes irrelevant terms, and selects
the top 3000 most frequent words to be used as features for the Naive Bayes model.


In [ ]:
def make_Dictionary(root_dir):
    # Initialize an empty list to store all words from the emails
    all_words = []

    # Create a list of full file paths for all email files in the directory
    emails = [os.path.join(root_dir, f) for f in os.listdir(root_dir)]

    # Read each email file
    for mail in emails:
        with open(mail) as m:
            # Read the email line by line
            for line in m:
                # Split each line into individual words
                words = line.split()
                # Add the words to the overall word list
                all_words += words

    # Count the frequency of each word
    dictionary = Counter(all_words)

    # Create a list of dictionary keys for cleaning
    list_to_remove = list(dictionary)

    # Remove non-alphabetic words and single-character words
    for item in list_to_remove:
        if not item.isalpha():
            del dictionary[item]
        elif len(item) == 1:
            del dictionary[item]

    # Select the 3000 most common words as features
    dictionary = dictionary.most_common(3000)

    # Return the final dictionary
    return dictionary

The above code builds a vocabulary of the most frequent words from the training emails to be used as features in the Naive Bayes model.

## Conceptual Analysis

The purpose of this function is to turn raw email text into usable features for a **Machine Learning Model.** Unfortuately, MLM only undertsand numbers. Therefore we decide which words matter and represent emails using those words.





1.   Collect all words from training emails
*   The function reads every email in the training folder and extracts every word.
*   Prequisite: What words appear in the dataset, How often each words appears.
*   Output: a global view of the language used in spam and non-spam emails.




2.   Using `Counter`, the function counts how many times each word occurs across all training emails.

In spam detection:
- Words that appear frequently (e.g., free, win, money) are often informative
- Rare or random words are usually not useful




3. Remove uninformative words
- The function removes:
Words with numbers or symbols (isalpha() == False) and
Single-character words (len(item) == 1)

### Why ?

- They do not carry meaningful semantic information
- They add noise instead of helping classification

###Output: This cleaning step improves model accuracy and efficiency.



4. Keep only the most frequent words (3000 words)

Why limit the size?
- Too many features make the model slower
- Many rare words add noise
- Naive Bayes works best with a controlled feature set

This step balances:
**Information richness **vs.** model simplicity**

5. Return back to dictionary (Vocabulary)

The final output is a list of the **3000 most frequent and meaningful words.**

This dictionary becomes the feature space:
- Each word = one feature
- Each email will later be represented by counts of these words

## Summary
The make_Dictionary function builds a vocabulary from the training emails by collecting all words, counting their frequencies, removing irrelevant terms, and selecting the most common words. This vocabulary defines the feature space used to convert raw email text into numerical representations that can be processed by the Naive Bayes classifier.

## Extracting Features from Emails

This function converts each email into a **numerical feature vector** using the previously created dictionary.
Each email is represented by word frequencies, which are later used to train the Naive Bayes classifier.


In [ ]:
def extract_features(mail_dir):
    # Generate full file paths for all email files in the directory
    files = [os.path.join(mail_dir, fi) for fi in os.listdir(mail_dir)]

    # Initialize the feature matrix (rows: emails, columns: dictionary words)
    features_matrix = np.zeros((len(files), 3000))

    # Initialize the label vector (0 = non-spam, 1 = spam)
    train_labels = np.zeros(len(files))

    # Initialize counters
    count = 1
    docID = 0

    # Loop through each email file
    for fil in files:
        # Open the email file
        with open(fil) as fi:
            # Read the file line by line
            for i, line in enumerate(fi):
                # Process only the third line, which contains the email content
                if i == 2:
                    # Split the line into individual words
                    words = line.split()

                    # Loop through each word in the email
                    for word in words:
                        # Search for the word in the dictionary
                        for idx, d in enumerate(dictionary):
                            if d[0] == word:
                                # Store the word frequency in the feature matrix
                                features_matrix[docID, idx] = words.count(word)

        # Assign default label as non-spam (0)
        train_labels[docID] = 0

        # Extract the file name from the file path
        filepathTokens = fil.split('/')
        lastToken = filepathTokens[-1]

        # Assign label as spam (1) if file name starts with "spmsg"
        if lastToken.startswith("spmsg"):
            train_labels[docID] = 1
            count += 1

        # Move to the next document index
        docID += 1

    # Return the feature matrix and corresponding labels
    return features_matrix, train_labels


# Conceptual Thinking and Startegies

The overall goal is to build an email spam classifier using Naive Bayes.
Startegi preview:

- Build a vocabulary (dictionary) from training emails
- Convert each email into a feature vector based on that vocabulary
- Assign labels (spam vs non-spam) for supervised learning
- Train Multinomial Naive Bayes on those features
- Predict on test emails and evaluate using accuracy (and optionally confusion matrix)

Multinomial Naive Bayes is designed for count-based features (word frequencies).

It learns:
- Probability of a word given spam: P(word | spam)
- Probability of a word given ham: P(word | ham)

Then for a new email, it computes:
- How likely the email is spam vs ham based on the words it contains

Naive Bayes proven effective as it:
- Very fast for text classification
- Works well with high-dimensional sparse data (many word features)
- Strong baseline model for spam detection

##Summary

The code implements a classic text-classification pipeline. It first builds a vocabulary from training emails by selecting the most frequent meaningful words. Then it converts each email into a bag-of-words feature vector (word frequency counts) and assigns labels based on the file naming convention. These features and labels allow a Multinomial Naive Bayes model to learn word probability patterns for spam versus non-spam emails, enabling prediction and evaluation on unseen messages.

## Data Paths

Purpose:

This cell sets the directory paths for the training and testing email folders. Adjust these paths if your folder structure is different.

In [ ]:
# Enter the paths of the training and testing folders
# Update these if your data is stored elsewhere
TRAIN_DIR = "./test-mails"
TEST_DIR = "./train-mails"

## Dataset Paths

This section specifies the directory paths for the training and testing email datasets.

Using relative paths ensures that the notebook can be executed consistently across different environments without modifying the code.

In [ ]:
print("TRAIN_DIR =", TRAIN_DIR)
print("TEST_DIR  =", TEST_DIR)

TRAIN_DIR = ./test-mails
TEST_DIR  = ./train-mails


In [ ]:
dictionary = make_Dictionary(TRAIN_DIR)

In [ ]:
TRAIN_DIR = "./train-mails"
TEST_DIR  = "./test-mails"

print("TRAIN_DIR =", TRAIN_DIR)
print("TEST_DIR  =", TEST_DIR)

TRAIN_DIR = ./train-mails
TEST_DIR  = ./test-mails


In [ ]:
dictionary = make_Dictionary(TRAIN_DIR)

## Verifying Dataset Paths

This step prints the training and testing directory paths to verify that the correct folders are being used before feature extraction and model training.

print() is used to verify that the correct dataset paths are assigned before building the dictionary and extracting features.

## Build Feature Matrices

Purpose:

This cell builds the vocabulary using only the training data, then converts both the training and test emails into numeric feature matrices that share the same column order.

Key idea:

The dictionary must be created from training data only to avoid data leakage, and the same dictionary is used for both train and test so the model sees consistent features.

In [ ]:
# Build the dictionary from training emails only
dictionary = make_Dictionary(TRAIN_DIR)

# Convert emails to feature vectors using the same dictionary
print ("reading and processing emails from TRAIN and TEST folders")
features_matrix, labels = extract_features(TRAIN_DIR)
test_features_matrix, test_labels = extract_features(TEST_DIR)

reading and processing emails from TRAIN and TEST folders


# Train and Evaluate the Model

Purpose:

This cell trains a Multinomial Naive Bayes classifier on the training feature matrix, predicts labels for the test set, and prints the accuracy score.

Why Multinomial Naive Bayes:

The features are word counts, so MultinomialNB is a natural choice because it models counts directly.

In [ ]:
# Step 1: Build the dictionary (vocabulary) from training emails
print("reading and processing emails from TRAIN and TEST folders")

dictionary = make_Dictionary(TRAIN_DIR)

# Step 2: Extract features and labels from training and test sets
train_features, train_labels = extract_features(TRAIN_DIR)
test_features, test_labels = extract_features(TEST_DIR)

# Step 3: Train Naive Bayes model (MultinomialNB is suitable for word counts)
print("Training Model using Multinomial Naive Bayes algorithm .....")
model = MultinomialNB()
model.fit(train_features, train_labels)
print("Training completed")

# Step 4: Predict labels for test data
print("testing trained model to predict Test Data labels")
predicted_labels = model.predict(test_features)
print("Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:")

# Step 5: Evaluate performance
accuracy = accuracy_score(test_labels, predicted_labels)
print(accuracy)

reading and processing emails from TRAIN and TEST folders
Training Model using Multinomial Naive Bayes algorithm .....
Training completed
testing trained model to predict Test Data labels
Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:
0.9615384615384616


## Training and Evaluating the Naive Bayes Model

Finally in this section, the dictionary is built using the training emails, then both training and testing emails are converted into numerical feature vectors (word frequency counts).  
A Multinomial Naive Bayes model is trained using the training features and labels. Finally, the trained model predicts labels for the test set, and the accuracy score is computed by comparing predicted labels with true test labels.


======================= END OF PROGRAM =========================